# Analyzing Crime and Education Data in Chicago

In [1]:
import pandas as pd
import sqlite3

In [2]:
census_data = pd.read_csv('ChicagoCensusData.csv')
public_schools = pd.read_csv('ChicagoPublicSchools.csv')
crime_data = pd.read_csv('ChicagoCrimeData.csv')

In [3]:
conn = sqlite3.connect('FinalDB.db')

census_data.to_sql('CENSUS_DATA', conn, if_exists='replace', index=False)
public_schools.to_sql('CHICAGO_PUBLIC_SCHOOLS', conn, if_exists='replace', index=False)
crime_data.to_sql('CHICAGO_CRIME_DATA', conn, if_exists='replace', index=False)

533

In [6]:
%load_ext sql
%sql sqlite:///FinalDB.db

### Problem 1
Find the total number of crimes recorded in the CRIME table.

In [8]:
%%sql

SELECT COUNT(*) as Total_Crimes
FROM CHICAGO_CRIME_DATA;


 * sqlite:///FinalDB.db
Done.


Total_Crimes
533


### Problem 2
List community areas with per capita income less than 11000.

In [9]:
%%sql
SELECT community_area_name, per_capita_income
FROM CENSUS_DATA
WHERE per_capita_income < 11000;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,PER_CAPITA_INCOME
West Garfield Park,10934
South Lawndale,10402
Fuller Park,10432
Riverdale,8201


### Problem 3
List all case numbers for crimes  involving minors?(children are not considered minors for the purposes of crime analysis)

In [10]:
%%sql
SELECT case_number
FROM CHICAGO_CRIME_DATA
WHERE description LIKE '%MINOR%';

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER
HL266884
HK238408


### Problem 4
List all kidnapping crimes involving a child?

In [11]:
%%sql
SELECT case_number
FROM CHICAGO_CRIME_DATA
WHERE primary_type = 'KIDNAPPING'
    AND description LIKE '%CHILD%';

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER
HN144152


### Problem 5
What kind of crimes were recorded at schools?

In [12]:
%%sql
SELECT DISTINCT(primary_type)
FROM CHICAGO_CRIME_DATA
WHERE location_description LIKE '%SCHOOL%';

 * sqlite:///FinalDB.db
Done.


PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


### Problem 6
List the average safety score for all types of schools.

In [13]:
%%sql
SELECT "Elementary, Middle, or High School" as school_type, AVG(safety_score) as avg_safety_score
FROM CHICAGO_PUBLIC_SCHOOLS
GROUP BY school_type;

 * sqlite:///FinalDB.db
Done.


school_type,avg_safety_score
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


### Problem 7
List 5 community areas with highest % of households below poverty line.

In [14]:
%%sql
SELECT community_area_name, percent_households_below_poverty
FROM CENSUS_DATA
ORDER BY percent_households_below_poverty DESC
LIMIT 5;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


### Problem 8
Which community area(number) is most crime prone?

In [15]:
%%sql
SELECT community_area_number, COUNT(*) as num_crimes
FROM CHICAGO_CRIME_DATA
GROUP BY community_area_number
ORDER BY num_crimes DESC
LIMIT 1;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,num_crimes
25.0,43


### Problem 9
Use a sub-query to find the name of the community area with highest hardship index.

In [17]:
%%sql
SELECT community_area_name, hardship_index
FROM census_data
WHERE hardship_index = (SELECT MAX(hardship_index) FROM census_data);

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,HARDSHIP_INDEX
Riverdale,98.0


### Problem 10
Use a sub-query to determine the Community Area Name with most number of crimes?

In [19]:
%%sql
SELECT Community_Area_Name, COUNT(*) as Crime_Count
FROM CHICAGO_CRIME_DATA
JOIN CENSUS_DATA ON CHICAGO_CRIME_DATA.Community_Area_Number = CENSUS_DATA.Community_Area_Number
GROUP BY CHICAGO_CRIME_DATA.Community_Area_Number
ORDER BY COUNT(*) DESC
LIMIT 1;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,Crime_Count
Austin,43
